In [ ]:
import numpy as np
import pandas as pd 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [ ]:
test = pd.read_csv('/kaggle/input/skillbox-computer-vision-project/sample_submission.csv')
test.head()

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("/kaggle/input/model-new-v4/model.h5")

In [ ]:
predict_dict = {0:'anger',1:'contempt',2:'disgust',3:'fear',4:'happy',5:'neutral',6:'sad',7:'surprise',8:'uncertain'}

In [ ]:
from keras import backend as K

def preprocess_input(x, data_format=None, version=1):
    x_temp = np.copy(x)
    if data_format is None:
        data_format = K.image_data_format()
    assert data_format in {'channels_last', 'channels_first'}

    if version == 1:
        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 93.5940
            x_temp[:, 1, :, :] -= 104.7624
            x_temp[:, 2, :, :] -= 129.1863
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 93.5940
            x_temp[..., 1] -= 104.7624
            x_temp[..., 2] -= 129.1863

    elif version == 2:
        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 91.4953
            x_temp[:, 1, :, :] -= 103.8827
            x_temp[:, 2, :, :] -= 131.0912
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 91.4953
            x_temp[..., 1] -= 103.8827
            x_temp[..., 2] -= 131.0912
    else:
        raise NotImplementedError

    return x_temp

In [ ]:
def preprocess_input_facenet(image_):

    preprocessed = preprocess_input(image_, version=2)
    
    return preprocessed

image_gen = ImageDataGenerator(preprocessing_function=preprocess_input_facenet) 

In [ ]:
BATCH_SIZE = 128
IMAGE_SIZE = 224

data = image_gen.flow_from_dataframe(
    test,
    directory='/kaggle/input/test-data/test_kaggle',
    x_col='image_path',
    y_col='emotion',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False)


In [ ]:
emotion = model.predict(data)
emotion_cm = [predict_dict[np.argmax(i)] for i in emotion]

In [ ]:
test['emotion'] = emotion_cm
test.head()

In [ ]:
test.to_csv('/kaggle/working/submission.csv', sep=',', index=False)

С этого момента было 17 ноутбуков и они +- одинаковые. Отличались только предобученные модели. Я исправил тестовый датасет, теперь shuffle = True и он образуеться от датафрейма. Также я изменил спооб искусственного увеличения датасета для обучения. Я увеличил изначальный датасет в 14 раз с помощью библеотеки "albumentations":

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

import albumentations as A

from PIL import Image

from os import listdir

import tensorflow as tf

import numpy as np

transform_1 = A.Compose([
            A.RandomCrop(width=224, height=224),
            A.HorizontalFlip(p=1),
            A.RandomBrightnessContrast(p=1),
        ])

transform_2 = A.Compose([
            A.RandomCrop(width=224, height=224),
            A.Blur(blur_limit=6, always_apply=True),
        ])

transform_3 = A.Compose([
            A.RandomCrop(width=224, height=224),
            A.ChannelDropout (channel_drop_range=(1, 1), fill_value=0, p=1),
        ])

transform_4 = A.Compose([
            A.RandomCrop(width=224, height=224),
            A.CLAHE (clip_limit=4.0, tile_grid_size=(8, 8), p=1),
        ])

transform_5 = A.Compose([
            A.RandomCrop(width=224, height=224),
            A.ColorJitter (brightness=1, contrast=1, saturation=1, hue=1, p=1),
        ])

transform_6 = A.Compose([
            A.RandomCrop(width=224, height=224),
            A.Downscale (scale_min=0.25, scale_max=0.25, interpolation=None, p=1),
        ])

transform_7 = A.Compose([
            A.RandomCrop(width=224, height=224),
            A.Emboss (alpha=(0.2, 0.5), strength=(0.2, 0.7), p=1),
        ])

transform_8 = A.Compose([
            A.RandomCrop(width=224, height=224),
            A.Equalize (mode='cv', by_channels=True, mask=None, mask_params=(), p=1),
        ])

transform_9 = A.Compose([
            A.RandomCrop(width=224, height=224),
            A.FancyPCA (alpha=1, p=1),
        ])

transform_10 = A.Compose([
            A.RandomCrop(width=224, height=224),
            A.GaussNoise (var_limit=(100.0, 500.0), mean=0, per_channel=True,p=1),
        ])

transform_11 = A.Compose([
            A.RandomCrop(width=224, height=224),
            A.GlassBlur (sigma=0.7, max_delta=4, iterations=2, mode='fast', p=1),
        ])

transform_12 = A.Compose([
            A.RandomCrop(width=224, height=224),
            A.HueSaturationValue (hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=1),
        ])

transform_13 = A.Compose([
            A.RandomCrop(width=224, height=224),
            A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), always_apply=False, p=1),
        ])

def new_image(transform,image,i):

    transformed = transform(image=image)["image"]
    
    img = Image.fromarray(transformed, 'RGB')
    
    img.save("D://Загрузки//train//train//" + i1 + '//+'+ str(i)+ '_' + i2)

def new_image_1(transform,image,i):

    transformed = transform(image=image)["image"]
    
    img = Image.fromarray(transformed, 'RGB')
    
    img.save("D://Загрузки//train//train_1//" + i1 + '//+'+ str(i)+ '_' + i2)

for i1 in listdir('D://Загрузки//train//train'):

    for i2 in listdir('D://Загрузки//train//train//'+i1):

        img = tf.keras.preprocessing.image.load_img('D://Загрузки//train//train//'+i1+'//'+i2,
                                                      target_size=(256, 256))
        image = np.array(img)
        for i,transform in enumerate([transform_1,transform_2,transform_3,transform_4,transform_5,transform_6,transform_7,transform_8,transform_9,transform_10,transform_11,transform_12,transform_13]):
            new_image(transform,image,i)

for i1 in listdir('D://Загрузки//train//train_1'):

    for i2 in listdir('D://Загрузки//train//train_1//'+i1):
        img = tf.keras.preprocessing.image.load_img('D://Загрузки//train//train_1//'+i1+'//'+i2,
                                                      target_size=(256, 256))
        image = np.array(img)
        for i,transform in enumerate([transform_1,transform_2,transform_3,transform_4,transform_5,transform_6,transform_7,transform_8,transform_9,transform_10,transform_11,transform_12,transform_13]):
            new_image_1(transform,image,i)
            
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

При обучении я использовал разные модели:

№1

base_model = tf.keras.Model([vggface_model.input], vggface_model.get_layer("activation_49").output)

model = tf.keras.Sequential([
  base_model,
  
  Dropout(0.5),
  
  Conv2D(2048, (3, 3),activation='relu',padding='same'),
  
  Conv2D(2048, (3, 3),activation='relu',padding='same'),
  
  BatchNormalization(),
  
  MaxPooling2D(pool_size=(7, 7)),
  
  Dropout(0.5),
  
  Flatten(), # Конвертация в двумерный тензор
  
  Dense(256, activation='relu'),
  
  Dense(128, activation='relu'),
  
  BatchNormalization(),
  
  Dropout(0.2),
  
  Dense(9, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss="categorical_crossentropy",
              metrics=['accuracy'])
              
№2

base_model = tf.keras.Model([vggface_model.input], vggface_model.get_layer("conv5_3_3x3/bn").output)

model = tf.keras.Sequential([
  base_model,
  
  Conv2D(1024, (3, 3),activation='relu',padding='same'),
  
  Conv2D(1024, (3, 3),activation='relu',padding='same'),
  
  BatchNormalization(),
  
  MaxPooling2D(pool_size=(2, 2)),
  
  Dropout(0.5),
  
  Conv2D(2048, (3, 3),activation='relu',padding='same'),
  
  Conv2D(2048, (3, 3),activation='relu',padding='same'),
  
  BatchNormalization(),
  
  Dropout(0.5),
  
  Flatten(),
  
  Dense(256, activation='relu'),
  
  Dense(128, activation='relu'),
  
  BatchNormalization(),
  
  Dropout(0.2),
  
  Dense(9, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss="categorical_crossentropy",
              metrics=['accuracy'])
              
№3

base_model = tf.keras.Model([vggface_model.input], vggface_model.get_layer("flatten_1").output)

model = tf.keras.Sequential([
  base_model,
  
  tf.keras.layers.Dense(1024, activation='relu'),
  
  tf.keras.layers.Dense(512, activation='relu'),
  
  tf.keras.layers.Dense(256, activation='relu'),
  
  tf.keras.layers.Dense(128, activation='relu'),
  
  tf.keras.layers.Dense(64, activation='relu'),
  
  tf.keras.layers.Dense(32, activation='relu'),
  
  tf.keras.layers.Dense(16, activation='relu'),
  
  tf.keras.layers.Dense(9, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss="categorical_crossentropy",
              metrics=['accuracy'])
              
№4

model = Sequential([
    Conv2D(64, (5, 5), input_shape=(48,48,3), activation='relu', padding='same'),
    
    Conv2D(64, (5, 5), activation='relu', padding='same'),
    
    BatchNormalization(),
    
    MaxPooling2D(pool_size=(2, 2)),
    
    Dropout(0.5),

    Conv2D(128, (5, 5),activation='relu',padding='same'),
    
    Conv2D(128, (5, 5),activation='relu',padding='same'),
    
    BatchNormalization(),
    
    MaxPooling2D(pool_size=(2, 2)),
    
    Dropout(0.5),

    Conv2D(256, (3, 3),activation='relu',padding='same'),
    
    Conv2D(256, (3, 3),activation='relu',padding='same'),
    
    BatchNormalization(),
    
    MaxPooling2D(pool_size=(2, 2)),
    
    Dropout(0.5),

    Flatten(),
    
    Dense(128, activation='relu'),
    
    BatchNormalization(),
    
    Dropout(0.2),
    
    Dense(9, activation='softmax')
])

my_optimiser = tf.keras.optimizers.Adam(
                    learning_rate=0.001, 
                    beta_1=0.9, 
                    beta_2=0.999, 
                    epsilon=1e-07, 
                    amsgrad=False,
                    name='Adam')

model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=my_optimiser)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Лучшей моделью оказалась №3. При обучении моделей с 1-3, они начинали переобучаться на 2 эпохе.

Таким образом лучшего результата получилось достичь при увеличении датасета в 14 раз с помощью библиотеки "albumentations". Также при разбиении данных на тестовые и валидационные 9к1.
И использовании модели под №3. В итоге Score на тестовых данных: 0.5252.